In [1]:
import numpy as np
import json
import geog
import shapely.geometry
from shapely.geometry import Point, shape
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
from operator import itemgetter
import glob
import json
from ast import literal_eval
import osmnx as ox


In [2]:
stations_df = pd.read_csv("asemat.csv")
stations_df = gpd.GeoDataFrame(stations_df, geometry=gpd.points_from_xy(stations_df.X,stations_df.Y))


In [3]:
services_df = pd.read_csv("palvelut.csv")
services_df.location = services_df.location.apply(literal_eval)
services_df.location = services_df.location.apply(lambda x: Point(x['coordinates'][0], x['coordinates'][1]))
services_df = gpd.GeoDataFrame(services_df, geometry='location')
typesr = []
services_df['type'] = services_df['type'].apply(lambda x: literal_eval(x))
services_df['type'].apply(lambda x: typesr.append(x))
types = [t for t_ in typesr for t in t_]
types = set(types)
types

{'A1-englanti (perusopetuksen kieliohjelma)',
 'A1-saksa (perusopetuksen kieliohjelma)',
 'A2-englanti (perusopetuksen kieliohjelma)',
 'B1-ruotsi (perusopetuksen kieliohjelma)',
 'Kelan etuudet',
 'aikuisten sosiaaliohjaus',
 'aikuisten sosiaalityö',
 'aikuisten terveyskeskuspäivystys ja päivystyspoliklinikka',
 'ajoneuvojen siirrot',
 'alueellinen kumppanuustyö',
 'ammatillinen lisäkoulutus',
 'ammatillinen peruskoulutus',
 'ammatilliseen peruskoulutukseen valmentava koulutus - VALMA',
 'apteekit',
 'asemakaava-arkisto',
 'asiakaspalvelu (energia)',
 'asiakaspalvelu (joukkoliikenne)',
 'asiakaspalvelu (kaavoitus)',
 'asiakaspalvelu (työväenopisto)',
 'asiakaspalvelu ja neuvonta (ammatillinen koulutus)',
 'asiakaspalvelu ja neuvonta (lukiokoulutus)',
 'asiakaspalvelu ja neuvonta (perusopetus)',
 'asiakaspalvelu ja neuvonta (sosiaali- ja terveyspalvelut)',
 'askartelu (nuorten harrastustila)',
 'asukaspysäköintitunnukset',
 'asumisen tuet',
 'asuntotonttien vuokraus ja myynti',
 'avioe

In [4]:
# Take only the commercial services from the df
comtypes = ['apteekit', 'baarit ja yöelämä','elokuvateatterit',
            'hotellit ja majoitus','kahvilat','kaupat ja myymälät',
            'kioskit',  'lipunmyynti (tapahtumat)', 'ostospaikat', 'ravintolat','torit']

mask = services_df['type'].apply(lambda x: any(c for c in comtypes if c in x))
com_df = services_df[mask]
# Take all traffic nodes from the df
traftypes = [ 'joukkoliikennepysäkit', 'metroasemat', 'rautatieasemat', 'yleiset pysäköintialueet']
mask = services_df['type'].apply(lambda x: any(t for t in traftypes if t in x))
traf_df = services_df[mask]


In [5]:
# Cluster the commercial services
nclusters = 300
coords = com_df.location.apply(lambda x: [x.x, x.y])
coords = np.array(list(coords))
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = nclusters, n_init=10)
kmeans.fit(coords)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=300, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [6]:
# Get all the clusters into a GeoDataFrame with coordinates and cluster size
clusters = {}
for label in kmeans.labels_:
    if str(label) not in clusters:
        clusters[str(label)] = {}
        clusters[str(label)]['label'] = int(label)
        clusters[str(label)]['x'] = float(kmeans.cluster_centers_[label][1])
        clusters[str(label)]['y'] = float(kmeans.cluster_centers_[label][0])
        clusters[str(label)]['n'] = 1
    else:
        clusters[str(label)]['n'] += 1

clusters_df= pd.DataFrame.from_dict(clusters, orient='index')
clusters_df

clusters_gdf = gpd.GeoDataFrame(clusters_df, geometry=gpd.points_from_xy(clusters_df.y,clusters_df.x))

In [7]:
clusters_gdf

,label,x,y,n,geometry
20,20,60.166597,24.945503,9,POINT (24.94550 60.16660)
172,172,60.183368,24.924370,4,POINT (24.92437 60.18337)
82,82,60.229282,24.881021,1,POINT (24.88102 60.22928)
189,189,60.169978,24.927861,8,POINT (24.92786 60.16998)
281,281,60.166169,24.943684,14,POINT (24.94368 60.16617)
...,...,...,...,...,...
63,63,60.175026,24.980251,2,POINT (24.98025 60.17503)
230,230,60.179810,24.982718,1,POINT (24.98272 60.17981)
214,214,60.175419,24.984305,2,POINT (24.98431 60.17542)
122,122,60.174936,24.988665,2,POINT (24.98867 60.17494)


In [8]:
def min_dist(point, gpd2):
    gpd2['Dist'] = gpd2.apply(lambda row:  point.distance(row.geometry),axis=1)
    geoseries = gpd2.iloc[gpd2['Dist'].argmin()]
    return geoseries

## Find the nearest commercial cluster of each bike station
stations_df['nearesr_comcluster'] = stations_df.apply(lambda row: min_dist(row.geometry,clusters_gdf)['label'] , axis=1)
stations_df

,X,Y,FID,data,name,coordinate,total_slot,operative,id,geometry,nearesr_comcluster
0,24.950293,60.155445,1,"{""name"":""001 Kaivopuisto"",""coordinates"":""60.15...",Kaivopuisto,"60.15544479382098,24.950292889690314",30,Yes,1,POINT (24.95029 60.15544),150
1,24.956347,60.160959,2,"{""name"":""002 Laivasillankatu"",""coordinates"":""6...",Laivasillankatu,"60.160959094315416,24.95634747175871",12,Yes,2,POINT (24.95635 60.16096),179
2,24.944927,60.158189,3,"{""name"":""003 Kapteeninpuistikko"",""coordinates""...",Kapteeninpuistikko,"60.1581892,24.9449274",16,Yes,3,POINT (24.94493 60.15819),204
3,24.941776,60.160986,4,"{""name"":""004 Viiskulma"",""coordinates"":""60.1609...",Viiskulma,"60.1609859,24.9417758",14,Yes,4,POINT (24.94178 60.16099),47
4,24.936285,60.157948,5,"{""name"":""005 Sepänkatu"",""coordinates"":""60.1579...",Sepänkatu,"60.1579483,24.9362853",32,Yes,5,POINT (24.93629 60.15795),0
...,...,...,...,...,...,...,...,...,...,...,...
340,24.831397,60.235089,341,"{""name"":""757 Painiitty"",""coordinates"":""60.2350...",Painiitty,"60.235089, 24.831397",12,Yes,757,POINT (24.83140 60.23509),37
341,24.839390,60.220467,342,"{""name"":""761 Mäkkylän asema"",""coordinates"":""60...",Mäkkylän asema,"60.220467, 24.839390",32,Yes,761,POINT (24.83939 60.22047),170
342,24.833536,60.220551,343,"{""name"":""763 Kalkkipellonmäki"",""coordinates"":""...",Kalkkipellonmäki,"60.220551,24.833536",24,Yes,763,POINT (24.83354 60.22055),37
343,24.820635,60.223377,344,"{""name"":""767 Ruutikatu"",""coordinates"":""60.2233...",Ruutikatu,"60.223377, 24.820635",10,Yes,767,POINT (24.82064 60.22338),37
